In [0]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import random
from torch.nn.utils.rnn import pad_sequence
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix

In [0]:
data_path = "drive/My Drive/NER/ner_dataset.csv"   # setting path for the data

In [0]:
df = pd.read_csv(data_path, encoding = "ISO-8859-1", error_bad_lines=False)
#reading the data as a pandas dataframe
df.head()
# The task is to map each Word to correct Tag

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O


In [0]:
# As the data has sentence id for the first word for the sentence only, and NaN for all other words
# We fill all the NaNs with the previous value.
# This is helpful later when we use groupby to extract data.
df = df.fillna(method='ffill')
df.head()

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,Sentence: 1,of,IN,O
2,Sentence: 1,demonstrators,NNS,O
3,Sentence: 1,have,VBP,O
4,Sentence: 1,marched,VBN,O


In [0]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 4 columns):
Sentence #    1048575 non-null object
Word          1048575 non-null object
POS           1048575 non-null object
Tag           1048575 non-null object
dtypes: object(4)
memory usage: 32.0+ MB


In [0]:
# Now we want to extract (Word, Tag) pairs for "each sentence".
# The following extract_pairs function will extract all the (Word , Tag) pairs regardless of the sentence.
# So we apply it to pandas groupby
def extract_pairs(df):
    return [(word.lower() , tag.lower()) for word, tag in zip(df["Word"].values.tolist() , df["Tag"].values.tolist())]


In [0]:
# We group by sentence # and then make pairs for each individual sentence.
sentences_pairs = df.groupby("Sentence #").apply(extract_pairs)

In [0]:
# So for first sentence the (Word,Tag) pairs look like this
sentences_pairs[0]

[('thousands', 'o'),
 ('of', 'o'),
 ('demonstrators', 'o'),
 ('have', 'o'),
 ('marched', 'o'),
 ('through', 'o'),
 ('london', 'b-geo'),
 ('to', 'o'),
 ('protest', 'o'),
 ('the', 'o'),
 ('war', 'o'),
 ('in', 'o'),
 ('iraq', 'b-geo'),
 ('and', 'o'),
 ('demand', 'o'),
 ('the', 'o'),
 ('withdrawal', 'o'),
 ('of', 'o'),
 ('british', 'b-gpe'),
 ('troops', 'o'),
 ('from', 'o'),
 ('that', 'o'),
 ('country', 'o'),
 ('.', 'o')]

In [0]:
# Now lets do some exploration 
# Counting unique words and making word2index index2word , tag2index, index2tag
def make_dicts(sentences_pairs):
    word2index = {}
    index2word = {}
    tag2index = {}
    index2tag = {}
    # We add speacial symbol <pad> in our word list
    word2index["<pad>"] = 0
    index2word[0] = "<pad>"
    word_index = 1
    # We add <BOT> and <EOT> symbols ie Beginning of tag and End Of Tag along with <PADT> ie Pad Tag
    tag2index["<PADT>"] = 0
    tag2index["<BOT>"]  = 1
    tag2index["<EOT>"] = 2
    index2tag[0] = "<PADT>"
    index2tag[1] = "<BOT>"
    index2tag[2] = "<EOT>"
    tag_index = 3
    # now indexing
    for sent in sentences_pairs:
        for pair in sent:
            word = pair[0]
            tag = pair[1]
            if(word not in word2index):
                word2index[word] = word_index
                index2word[word_index] = word
                word_index = word_index + 1
            if(tag not in tag2index):
                tag2index[tag] = tag_index
                index2tag[tag_index] = tag
                tag_index += 1
    return word2index , index2word , tag2index , index2tag
            


In [0]:
# gettting dictionaries, we will beusing this for all further indexing
word2index , index2word , tag2index , index2tag  =make_dicts(sentences_pairs)


In [0]:
# Here is how tag2indexlooks like....
tag2index

{'<BOT>': 1,
 '<EOT>': 2,
 '<PADT>': 0,
 'b-art': 13,
 'b-eve': 18,
 'b-geo': 4,
 'b-gpe': 5,
 'b-nat': 15,
 'b-org': 7,
 'b-per': 9,
 'b-tim': 6,
 'i-art': 14,
 'i-eve': 19,
 'i-geo': 8,
 'i-gpe': 16,
 'i-nat': 17,
 'i-org': 11,
 'i-per': 10,
 'i-tim': 12,
 'o': 3}

In [0]:
# here is how these dictionaries work
print(word2index["i"])
print(index2word[432])

4702
refused


In [0]:
# Converting all the words into their respective index
# So ("london" ,"b-eve") becomes (324 , 6)
def words_to_numbers(data):
    new_data = []
    for sent in data:
        new_sent = []
        for pair in sent:
            new_sent.append((word2index[pair[0]] ,tag2index[pair[1]]))
        new_data.append(new_sent)
    return new_data


In [0]:
sentences_pairs = words_to_numbers(sentences_pairs)

In [0]:
# Now lets split our test and train data
test_data_size = int(.2*len(sentences_pairs))
# Randomly Shuffling the data
random.shuffle(sentences_pairs)

In [0]:
test_data = np.array(sentences_pairs[:test_data_size])
train_data = np.array(sentences_pairs[test_data_size:])
# We convert into numpy array so that its easy to index them

In [0]:
# Next lets make a function that feeds batches to out model
# I use stostatic sampling here 
def get_train_batch(data , batch_size):
    index = np.random.randint(low = 0 , high = len(data) -1 , size = batch_size)
    batch = data[index].tolist()
    # We have a random sample of size "batch_size"
    batch = list(map(lambda x: torch.tensor(x) , batch)) # Converting each batch into tensors
    batch = pad_sequence(batch , batch_first= True) # We now pad the shorter sequences with zeros
    # batch has shape (batch_size , len of longest_seq , 2), the last dim is for words and tags
    # seperating words and tags
    X = batch[:,:,0]   # Input data
    y = batch[:,:,1]   # Target values
    return X.to(dtype=torch.long, device=device),y.to(dtype=torch.long, device=device)


***Now we can start writting the model***

In [0]:
# We start writting the CRF model first as I will be using it on top of the main LSTM model
# For now assume running LSTM model gives output of size (batch_size , len of longest seq , n_labels)
# Notice n_labels here is actually the total number of possible Tags in this case 20.

In [0]:
# I have posted links about the CRF linear model in the github project. 
# WE use CRF model to calculate the loss function
class CRF_linear(nn.Module):
    def __init__(self,  n_labels):
        super().__init__()
        # we first define the transition matrix
        # Tm[i,j] represents the probablity of going from i th tag to j th
        self.n_labels = n_labels 
        self.transition_matrix = nn.Parameter(torch.rand(self.n_labels , self.n_labels))
        # now lets set the probablity of going to any tag from <EOT> almost zero
        # similarly prob of going from any tag to <BOT>
    #    self.transition_matrix[tag2index["<EOT>"]:] = -10000.0
    #    self.transition_matrix[:tag2index["<BOT>"]] = -10000.0
        # Large negative number will zero the probablity as we will be using exp.
        # Will come back to this later.
    def forward(self, predicted , targets):  # Returns the loss
        # Predicted size --> (batch_size , len of longest seq , n_labels)
        # target size -->    (batch_size , len of longest seq)
        # We must now calculate the P(y/X) for this input.
        # This requires us to calculate both probablity of current alignment
        # and sum of probablity of all possible alignment (partition function)
        # Finally our loss is log of (P(current_alignment)/partition function)
        return self.partition_optimized(predicted, targets) - self.calc_current_score(predicted , targets)

    def calc_current_score(self ,predicted , targets):
        batch_size , seq_len , _ = predicted.size()
        # Two scores are computed here
        # Stand alone probablity score ie what is the probablity for the correct target
        # transition score


        # computing stand alone probablity score
        # gathering the probablity for the correct target.  
        prob_score = torch.gather(dim = 2 ,input = predicted , index = targets.unsqueeze(-1))
        prob_score =  prob_score.sum(1) # Has shape (batch_size , 1)
        prob_score = prob_score.squeeze() # shape (batch_size)

        # now computing the transition scores
        trans_score = torch.zeros(predicted.size(0)).float().to(device)
        # initialize with the transition form <BOT> to first target
        trans_score = trans_score  + self.transition_matrix[tag2index["<BOT>"], targets[:,0]]
        for i in range(1, seq_len):
            trans_score = trans_score + self.transition_matrix[targets[:,i-1] , targets[:,i]]

        trans_score = trans_score + self.transition_matrix[targets[:,-1] , tag2index["<EOT>"]]
        
        # Notice how i am not taking log here, I am already in log space and it has negated the exp
        return trans_score + prob_score 

    def calc_partition_func(self , predicted, targets):
        # Running the Forward algo
        batch_size , seq_len , _ = predicted.size()
        n_labels = self.n_labels 

        # alpha_iter at every time step calculates the probablity of reaching the given label through all paths
        # Remember this is in log space
        alpha_iter = predicted[:,0,:] + self.transition_matrix[tag2index["<BOT>"],:].unsqueeze(0)

        for i in range(1 , seq_len):  # Iterating over seq_length
            store = torch.empty(batch_size , n_labels).to(device)
            for j in range(n_labels): # Iterating over tags
                # Idea is to store probality of getting to the jth tag in the ith seq
                # Then use it to calculate the ith +1 seq
                prob_tag = predicted[:, i,j].unsqueeze(-1)


                # All paths to out tag 
                transition_to_tag = self.transition_matrix[:,j].unsqueeze(0)

                # Adding rather than multplying as we are in log space

                prob_i_j = prob_tag + transition_to_tag + alpha_iter
                store[:, j] = torch.logsumexp(prob_i_j , dim = 1)
            alpha_iter = store
        # Finally add the transition form any to <EOT>
        alpha_iter += self.transition_matrix[:,tag2index["<EOT>"]]
        return torch.logsumexp(alpha_iter , dim = 1)

    def partition_optimized(self, predicted , targets):  # optimized form of calc_partition_func
        # The difference here is that we perform all the computation in matrix format
        # This makes this code faster on a GPU
        batch_size , seq_len , _ = predicted.size()
        n_labels = self.n_labels 
     
        # I store all the alphas values as we need all of them to compute gradient
        # This is a miniute but important detail.
        alpha_iter = torch.empty(batch_size,seq_len, self.n_labels).to(device)
         
        alpha_iter[:,0,:] = predicted[:,0,:] + self.transition_matrix[tag2index["<BOT>"],:].unsqueeze(0)

        # We can finish all the computation in one loop only
        for i in range(1, seq_len):
            # This does exactely what the calc_partition_func does.
            store = self.transition_matrix.unsqueeze(0) + alpha_iter[:,i-1,:].unsqueeze(-1) + predicted[:,i,:].unsqueeze(1)
            alpha_iter[:,i,:] = torch.logsumexp(store , dim = 1)
        
        alpha_iter[:,seq_len -1, :] += self.transition_matrix[:,tag2index["<EOT>"]]
        return torch.logsumexp(alpha_iter[:,seq_len-1,:] , dim = 1)
      


In [0]:
# Now onto the main model (This will use the CRF model)

class Tagger(nn.Module):
    def __init__(self , n_labels, lstm_hidden_dim , embedding_dim , lstm_layers):
        super().__init__()
        self.n_labels = n_labels
        self.lstm_hidden_dim = lstm_hidden_dim
        self.embedding_dim = embedding_dim
        self.lstm_layers = lstm_layers
        # We need a trainable word embedding (Could have used pre trained also :))
        self.embedding = nn.Embedding(self.embedding_dim[0], self.embedding_dim[1])

        # Now initializing out Bi-LSTM layer
        self.LSTM = nn.LSTM(input_size= self.embedding_dim[1] , hidden_size= self.lstm_hidden_dim , batch_first = True , bidirectional = True , num_layers = lstm_layers)

        # We also need a linear layer to translate BiLSTMs retults to proper shape (probs for all tags)
        self.linear1 = nn.Linear(self.lstm_hidden_dim*2 , self.n_labels)

        # Now lets use the CRF class we worte before
        self.CRF = CRF_linear(self.n_labels)
    def forward(self, X):
        X = self.embedding(X)  # Shape of X becomes (bs , seq_len , embedding_dim)
        X, _ = self.LSTM(X)     # shape of X becomes (bs, seq_len , lstm_hidden_dim*2)
        X = self.linear1(X)   # shape of X becomes (bs , seq_len , n_labels)
        return X

    def loss(self, X,y):
        predictions = self.forward(X)
        loss = self.CRF(predictions , y)
        return loss
    
    def Infer(self, X):
    
        with torch.no_grad():
            predicted = self.forward(X)
            transition_matrix = self.CRF.transition_matrix.data
        batch_size , seq_len ,_ = predicted.size()
        # Applying the Viterbi algo for finding best path
        # We must maintain both alpha and a backpointer for this
        alpha_iter = predicted[:,0,:] + transition_matrix[tag2index["<BOT>"],:].unsqueeze(0)
        back_pointers = torch.empty(batch_size , seq_len -1 , self.n_labels , dtype= torch.long)


        # The following for loop calculates the best probablity back pointers.
        # This is an optimization so that maximum calculation are done in matrix format
        for i in range(1,seq_len):
            store = transition_matrix.unsqueeze(0) + alpha_iter.unsqueeze(-1) + predicted[:,i,:].unsqueeze(1)
            alpha_iter , back_pointers[: , i-1, :]      =  torch.max(store , dim = 1)
    
        alpha_iter += transition_matrix[:,tag2index["<EOT>"]]

        # Getting the last max score and the index corrosponding to it
        max_path_score ,max_last = torch.max(alpha_iter ,dim = 1)
        return self.get_best_path(back_pointers ,max_last ,batch_size , seq_len)

    def get_best_path(self, back_pointers , max_last , batch_size , seq_len):
        # Here we finally assemble the path/answer using the back pointers
        path = torch.empty(batch_size ,seq_len , dtype = torch.long)
        path[:,-1] = max_last
        for i in range(seq_len- 2 , -1 ,-1):
            path[:,i] = torch.gather(input = back_pointers[:,i,:] , dim =1 , index = path[:,i+1].unsqueeze(-1)).squeeze()

        return path



   

In [0]:
model = Tagger(n_labels= len(tag2index) , lstm_hidden_dim= 100 , embedding_dim= (len(word2index), 140) , lstm_layers= 2)
model = model.to(device)

In [0]:
learning_rate = .001
optimizer = torch.optim.Adam(list(model.parameters()), learning_rate)

In [0]:
# Now lets train the model
def train(model , epochs , batch_size):
    inner_loop_size = len(train_data)//batch_size
    for _ in range(epochs):
        loss_arr = []
        epoch_loss = 0
        for i in range(inner_loop_size):
            X,y = get_train_batch(train_data ,batch_size)
            loss = model.loss(X,y).mean()  
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            epoch_loss += loss.item()
        print(epoch_loss/inner_loop_size)
        loss_arr.append(epoch_loss/inner_loop_size)

In [137]:
train(model ,10 , 10) 

0.10050016147943476
0.08728446180024467
0.08599331826486914
0.08365354076271884
0.08023602541406688
0.08254415404744397
0.0728342134136172
0.07544207415248697
0.07019999147085987
0.06912186529461242


In [0]:
# Now lets take our model for a test run !!!
class test_data_feeder:
    """
    This class feeds the test_data example by example 
    """
    def __init__(self,test_data): 
        self.test_data = test_data
        self.current_index = 0
    def get_next(self):
        if(self.current_index >= len(test_data)):
            return None
        
        nxt = np.array(self.test_data[self.current_index])
        X_test = torch.tensor(nxt[:,0]).to(device)
        y_test = nxt[:,1]
        self.current_index = self.current_index + 1
        return X_test, y_test
    

In [0]:
# Lets see an example 
test_loader = test_data_feeder(test_data)

In [0]:
X_test,y_test = test_loader.get_next()

In [141]:
# First lets see the correct sequence of outputs
print(y_test.tolist())

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]


In [142]:
# Now lets see what my model predicts
model.Infer(X_test.unsqueeze(0))[0].numpy().tolist()

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

In [0]:
# Well Well what do you know the model correctely predicted the entire sequence !!!!!

**Lets put it all to test !!! **

In [0]:
predicted_array = []
correct_array = []
# now running the model for all the test examples !!!!
def tester():
     while(True):
        obj = test_loader.get_next()
        if(obj == None):  # We we have finished iterating through the test data
            break 
        
        X_test , y_test = obj
        global predicted_array
        global correct_array
        predicted = model.Infer(X_test.unsqueeze(0))[0].numpy().tolist()
        predicted_array = predicted_array +  predicted
        correct_array += y_test.tolist()


In [0]:
tester()  # Running tester will fill the predicted and correct arrays

In [0]:
f1 = f1_score(correct_array, predicted_array, average="macro")

In [147]:
f1

0.5898674736952072

In [0]:
precision = precision_score(correct_array, predicted_array, average="macro")

In [149]:
precision

0.6346489731214398